<a href="https://colab.research.google.com/github/WilliamPoe/CSCI-290/blob/main/notebooks/Naive_Bayes_Classifier_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import math
from scipy.stats import norm

In [2]:
def Naive_Bayes_Classifier(X, y, instance, numerical, categorical):

  priors = {}
  likelihoods = {}


  for clss in X[y].unique():
    data = X[X[y] == clss] # Create a dataset for the rows where the clss is True
    priors[clss] = len(data) / len(X) # Calculate the priors for the dataset it created
    likelihoods[clss] = {} # Makes a dictionary where the clss is the key for the likelihoods
    for cat in categorical: # Gets likelihoods for categorical features
      likelihoods[clss][cat] = data[cat].value_counts(normalize=True).to_dict()
    for num in numerical: # Gets likelihoods for numerical features
      likelihoods[clss][num] = norm( loc=data[num].mean(), scale = data[num].std())

    posterior = {} # Create a dictionary for calculating the posteriors

    for clss in priors:
      post = priors[clss] # Sets post equal to priors for that clss
      for cat in categorical: # Categorical features
        if cat in instance: # Checks if the cat is in the unseen instance
          if instance[cat] in likelihoods[clss][cat]:
            post *= likelihoods[clss][cat][instance[cat]] # Multiplies post by the likelihoods for the cat features and then sets the post equal to that
          else:
            post *= 0
        else:
          pass
      for num in numerical: # Numerical features
        post *= likelihoods[clss][num].pdf(instance[num]) # Multiplies post by the likelihoods for the num features and then sets the post equal to that

      posterior[clss] = post # Sets the posterior for that clss equal to the post

    return max(posterior, key=posterior.get)

In [3]:
# Dataset
df = pd.read_csv("https://github.com/WilliamPoe/CSCI-290/raw/main/Data/ad_click_dataset.csv") # Change this to new dataset
df.head()
# Tagret column in the dataset
target = "ad_position"
# Unseen Instance
instance = {'age': 44.0, 'device_type': 'Desktop', 'browsing_history': 'Shopping', 'gender': 'Male'} # Make new unseen instance
# Numerical features
numerical = []
# Categorical features
categorical = []

## Remove [2:-1] if you want to test on all columns ##
for attrib in df.columns[2:-1]:
  if df[attrib].dtype in ['float64', 'int64']:
    # Considers the column numerical if there are more than 10 unique values
    if len( df[attrib].unique() ) > 10:
      numerical.append(attrib)
  else:
    categorical.append(attrib)

Naive_Bayes_Classifier(df, target, instance, numerical, categorical)


'Top'